# EDA on the IMDB dataset

## Downloading the datasets

In [4]:
!mkdir -p imdb; \
curl -o imdb/title.basics.tsv.gz https://datasets.imdbws.com/title.basics.tsv.gz; \
curl -o imdb/title.akas.tsv.gz https://datasets.imdbws.com/title.akas.tsv.gz; \
curl -o imdb/title.crew.tsv.gz https://datasets.imdbws.com/title.crew.tsv.gz; \
curl -o imdb/title.episode.tsv.gz https://datasets.imdbws.com/title.episode.tsv.gz; \
curl -o imdb/title.principals.tsv.gz https://datasets.imdbws.com/title.principals.tsv.gz; \
curl -o imdb/title.ratings.tsv.gz https://datasets.imdbws.com/title.ratings.tsv.gz; \
curl -o imdb/name.basics.tsv.gz https://datasets.imdbws.com/name.basics.tsv.gz; \
gunzip imdb/*.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  188M  100  188M    0     0  10.9M      0  0:00:17  0:00:17 --:--:-- 14.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  418M  100  418M    0     0  8207k      0  0:00:52  0:00:52 --:--:-- 4653k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70.1M  100 70.1M    0     0  8706k      0  0:00:08  0:00:08 --:--:-- 12.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  7279k      0  0:00:06  0:00:06 --:--:-- 10.6M
  % Total    % Received % Xferd  Average Speed   Tim

In [45]:
!mkdir -p movielens; \
curl -L -o movielens/movielens-dataset.zip https://www.kaggle.com/api/v1/datasets/download/grouplens/movielens-20m-dataset; \
unzip movielens/movielens-dataset.zip -d movielens

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  195M  100  195M    0     0  4943k      0  0:00:40  0:00:40 --:--:-- 3275k     0  0:00:37  0:00:31  0:00:06 4678k
Archive:  movielens/movielens-dataset.zip
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              
  inflating: tag.csv                 


I also opted to download the more updated (2023) version of the MovieLens dataset from `https://grouplens.org/datasets/movielens/` 

In [ ]:
!mkdir -p movielens32m; \
curl -L -o movielens32m/movielens-dataset.zip https://files.grouplens.org/datasets/movielens/ml-32m.zip ; \
unzip movielens32m/movielens-dataset.zip -d movielens32m

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 58  227M   58  134M    0     0   145k      0  0:26:48  0:15:47  0:11:01  112k 0  0:27:54  0:01:41  0:26:13  127k0  0:28:33  0:03:19  0:25:14  162kk      0  0:32:30  0:04:33  0:27:57 60196  119k      0  0:32:32  0:04:42  0:27:50  121k 0:27:29  0:08:04  0:19:25  264kk      0  0:26:50  0:15:30  0:11:20  220k

Run `ingest_datasets.py` first. This will store the datasets into a DuckDB persistent store.

Next run `profile_datasets.py` to produce HTML files containing analysis on data quality and profiles of each table. I have provided the outputs I generated in the `profiling` folder. Note that I set a limit of 10M records for profiling, as some of the datasets have more than 10M rows, which my PC could not handle.

## Data cleaning and exploration

We list below some notable findings from the data profiling.


name_basics
- majority (>90%) have missing birthYear
- a significant number (19%) have missing primaryProfession, most common: actor/actress
- knownForTitles and primaryProfessions are lists of values that can be normalized


title_basics
- more than 75% of the titles are TV episodes
- a significant number (12.5%) have missing startYear - a value that is important
- most (68%) do not have runtimeMinutes - it is less important but could be valuable
- genres still needs to be normalized

title_akas
- this table contains a list of alternate release versions for each unique title id from title_basics
- the attributes and types columns have a hex value '\x02' which is ";", they need to be replaced with comma then normalized

title_principals
- majority (>80%) have missing job value, and many have missing characters value
- from the whole dataset, 29M out of 89M rows have both missing job and characters - they only have category

title_ratings
- the averageRating varies around a median of 7.1
- the numVotes is heavily skewed where a majority of titles have low values (median is only 26)

In [1]:
import pandas as pd
import duckdb
db = duckdb.connect('imdb.duckdb')

In [3]:
db.close()

In [15]:
db.execute('select distinct attributes from title_akas_raw').fetchdf().values

array([['informal literal title'],
       ['8mm release title'],
       ['orthographically correct title'],
       ['bowdlerized title'],
       ['teaser title'],
       ['sixteenth season title'],
       ['last season title'],
       ['thirtysixth season title'],
       ['YIVO translation'],
       ['late Sunday edition'],
       ['fourth season title\x02recut version'],
       ['thirtieth season title'],
       ['R-rated version'],
       ['approximation of original mirrored title'],
       ['reissue title\x02Yiddish dubbed'],
       ['modern translation'],
       ['thirteenth season title\x02promotional title'],
       ['IMAX version\x02promotional title'],
       ['title for episodes with guest hosts'],
       ['reissue title\x02YIVO translation'],
       ['bootleg title\x02X-rated version'],
       ['poster title\x02video box title'],
       ['long title'],
       ['complete title'],
       ['third part title'],
       ['TV listings title'],
       ['informal alternative title'],


In [2]:
db.execute('select distinct types from title_akas_raw').fetchdf().values

array([['imdbDisplay\x02working'],
       ['alternative\x02video'],
       ['working\x02alternative'],
       ['working\x02festival'],
       ['alternative\x02festival'],
       [None],
       ['imdbDisplay'],
       ['imdbDisplay\x02tv'],
       ['imdbDisplay\x02festival'],
       ['original'],
       ['tv'],
       ['video'],
       ['alternative\x02tv'],
       ['working'],
       ['dvd'],
       ['working\x02tv'],
       ['working\x02video'],
       ['dvd\x02video'],
       ['alternative'],
       ['imdbDisplay\x02dvd'],
       ['festival'],
       ['imdbDisplay\x02video'],
       ['tv\x02video'],
       ['dvd\x02alternative']], dtype=object)

In [17]:
db.execute("""
    CREATE TABLE title_akas AS 
    SELECT 
        titleId, ordering, title, region, language,
        string_to_array(replace(types, '\x02', ';'), ';') as types,
        string_to_array(replace(attributes, '\x02', ';'), ';') as attributes,
        isOriginalTitle
    FROM title_akas_raw
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
db.execute('describe title_akas').fetchdf()

,column_name,column_type,null,key,default,extra
0,titleId,VARCHAR,YES,None,None,None
1,ordering,BIGINT,YES,None,None,None
2,title,VARCHAR,YES,None,None,None
3,region,VARCHAR,YES,None,None,None
4,language,VARCHAR,YES,None,None,None
5,types,VARCHAR[],YES,None,None,None
6,attributes,VARCHAR[],YES,None,None,None
7,isOriginalTitle,BIGINT,YES,None,None,None


In [ ]:
remove principals copy

In [ ]:
data cleaning
title_basics
primary title missing 18

In [ ]:
tbasics = pd.read_csv('imdb/title.basics.tsv', sep='\t', quoting=csv.QUOTE_NONE, na_values='\\N')

In [38]:
tbasics.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11272947 entries, 0 to 11272946
Data columns (total 9 columns):
 #   Column          Non-Null Count     Dtype  
---  ------          --------------     -----  
 0   tconst          11272947 non-null  object 
 1   titleType       11272947 non-null  object 
 2   primaryTitle    11272928 non-null  object 
 3   originalTitle   11272928 non-null  object 
 4   isAdult         11272947 non-null  int64  
 5   startYear       9856455 non-null   float64
 6   endYear         132734 non-null    float64
 7   runtimeMinutes  3561416 non-null   float64
 8   genres          10773067 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 774.1+ MB


In [40]:
tbasics[tbasics["startYear"]>2024].sample(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9457687,tt5924560,movie,Gray's Disconnect,Gray's Disconnect,0,2025.0,NaN,65.0,"Crime,Drama,Thriller"
3410455,tt14477560,movie,The Second Casualty,The Second Casualty,0,2025.0,NaN,90.0,War
7192234,tt29928512,movie,Blood Behind Us,Blood Behind Us,0,2025.0,NaN,NaN,"Action,Drama,Western"
7687890,tt32135710,movie,Bromance,Bromance,0,2025.0,NaN,NaN,NaN
2609870,tt13000882,tvEpisode,The Painter,The Painter,0,2025.0,NaN,NaN,"Drama,Fantasy,Horror"
7652503,tt32020189,movie,De La Cruz,De La Cruz,0,2025.0,NaN,128.0,Drama
8358659,tt34710193,tvEpisode,Episode #8.1,Episode #8.1,0,2025.0,NaN,NaN,NaN
1755373,tt11422710,tvMiniSeries,DaHaiGang,DaHaiGang,0,2025.0,NaN,NaN,Drama
7822665,tt32524691,tvSeries,Romantics Anonymous,Romantics Anonymous,0,2025.0,NaN,NaN,"Comedy,Drama,Romance"
6347902,tt26743210,movie,How to Train Your Dragon,How to Train Your Dragon,0,2025.0,NaN,NaN,"Action,Adventure,Comedy"


In [33]:
tbasics.startYear = tbasics.startYear.astype('Int64')

In [ ]:
tbasics = pd.read_csv('imdb/title.basics.tsv', sep='\t', quoting=csv.QUOTE_NONE)
tbasics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11272947 entries, 0 to 11272946
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         int64 
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: int64(1), object(8)
memory usage: 774.1+ MB


In [13]:
tbasics.titleType.value_counts()

titleType
tvEpisode       8659707
short           1030820
movie            699012
video            301285
tvSeries         273345
tvMovie          149006
tvMiniSeries      58171
tvSpecial         50453
videoGame         40673
tvShort           10474
tvPilot               1
Name: count, dtype: int64

In [11]:
sorted(tbasics.startYear.unique())

['1874',
 '1878',
 '1881',
 '1882',
 '1883',
 '1885',
 '1887',
 '1888',
 '1889',
 '1890',
 '1891',
 '1892',
 '1893',
 '1894',
 '1895',
 '1896',
 '1897',
 '1898',
 '1899',
 '1900',
 '1901',
 '1902',
 '1903',
 '1904',
 '1905',
 '1906',
 '1907',
 '1908',
 '1909',
 '1910',
 '1911',
 '1912',
 '1913',
 '1914',
 '1915',
 '1916',
 '1917',
 '1918',
 '1919',
 '1920',
 '1921',
 '1922',
 '1923',
 '1924',
 '1925',
 '1926',
 '1927',
 '1928',
 '1929',
 '1930',
 '1931',
 '1932',
 '1933',
 '1934',
 '1935',
 '1936',
 '1937',
 '1938',
 '1939',
 '1940',
 '1941',
 '1942',
 '1943',
 '1944',
 '1945',
 '1946',
 '1947',
 '1948',
 '1949',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 

In [25]:
tbasics.isAdult.unique()

array([0, 1])

In [21]:
!sed -n '7605952,7605959p' imdb/title.basics.tsv

tt31845330	tvEpisode	Episode #1.19	Episode #1.19	0	1997	\N	\N	History,Romance
tt31845331	tvEpisode	Episode #1.20	Episode #1.20	0	1997	\N	\N	History,Romance
tt31845333	tvEpisode	Episode #1.21	Episode #1.21	0	1997	\N	\N	History,Romance
tt31845334	tvEpisode	"Village of the Giants 65	"Village of the Giants 65	0	2019	\N	\N	Comedy,Drama,Fantasy
tt31845335	tvEpisode	Episode #1.56	Episode #1.56	0	2024	\N	\N	Comedy,Drama,Family
tt31845338	tvEpisode	Mr Sardonicus '61	Mr Sardonicus '61	0	2019	\N	\N	Comedy,Drama,Fantasy
tt3184534	videoGame	Valiant Hearts: The Great War	Valiant Hearts: The Great War	0	2014	\N	\N	Adventure,Drama,History
tt31845341	tvEpisode	S8 Ep33	S8 Ep33	0	2024	\N	\N	Talk-Show


In [26]:
tbasics.iloc[7605952:7605955]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
7605952,tt31845333,tvEpisode,Episode #1.21,Episode #1.21,0,1997,\N,\N,"History,Romance"
7605953,tt31845334,tvEpisode,"""Village of the Giants 65","""Village of the Giants 65",0,2019,\N,\N,"Comedy,Drama,Fantasy"
7605954,tt31845335,tvEpisode,Episode #1.56,Episode #1.56,0,2024,\N,\N,"Comedy,Drama,Family"


In [15]:
tbasics[~tbasics.isAdult.isin([0,1,'0','1'])].sample(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
7605953,tt31845334,tvEpisode,Village of the Giants 65\tVillage of the Giant...,0,2019,\N,\N,"Comedy,Drama,Fantasy",NaN
7904068,tt32779116,tvEpisode,Lost Planet Airmen\tLost Planet Airmen,0,1974,\N,\N,"Action,Adventure,Drama",NaN
7674820,tt32098928,tvEpisode,The Creation of the Humanoids\tThe Creation of...,0,1973,\N,\N,"Fantasy,Horror,Mystery",NaN
7771422,tt32377766,tvEpisode,Zontar the Thing from Venus'\tZontar the Thing...,0,1980,\N,\N,"Action,Horror,Mystery",NaN
7966053,tt33022175,tvEpisode,Track of the Vampire\tTrack of the Vampire,0,1985,\N,\N,"Fantasy,Horror,Mystery",NaN
7677078,tt32105765,tvEpisode,Voyage to the End of the Universe'\tVoyage to ...,0,1974,\N,\N,"Fantasy,Horror,Mystery",NaN
7913944,tt32820699,tvEpisode,The Mole People\tThe Mole People,0,1977,\N,\N,"Action,Adventure,Drama",NaN
7907353,tt32799017,tvEpisode,Lost Planet Airmen'\tLost Planet Airmen',0,1975,\N,\N,"Action,Adventure,Drama",NaN
7622004,tt31911836,tvEpisode,The Black Cat' 41\tThe Black Cat' 41,0,1983,\N,\N,"Fantasy,Horror,Mystery",NaN
7857241,tt32628047,tvEpisode,Frankenstein 1931\tFrankenstein 1931,0,1973,\N,\N,"Comedy,Drama,Horror",NaN


In [8]:
title_basics_df.isAdult.value_counts()

isAdult
0       10846635
1         360144
0          63527
1           2008
1985          79
1980          63
1978          54
1984          41
1982          32
1974          32
1972          29
2015          28
1983          25
2016          21
1977          20
1973          18
1976          18
1975          18
1981          18
2017          17
1986          17
1979          13
1987          13
2018           9
2020           9
2013           8
2019           7
2023           7
2014           6
1988           5
1966           5
2024           4
1968           3
1970           2
2022           2
1971           2
2011           1
\N             1
2012           1
2021           1
1969           1
1964           1
1967           1
2005           1
Name: count, dtype: int64

In [7]:
title_basics_df.groupby('isAdult').size()

isAdult
0       10846635
1         360144
1964           1
1966           5
1967           1
1968           3
1969           1
1970           2
1971           2
1972          29
1973          18
1974          32
1975          18
1976          18
1977          20
1978          54
1979          13
1980          63
1981          18
1982          32
1983          25
1984          41
1985          79
1986          17
1987          13
1988           5
2005           1
2011           1
2012           1
2013           8
2014           6
2015          28
2016          21
2017          17
2018           9
2019           7
2020           9
2021           1
2022           2
2023           7
2024           4
0          63527
1           2008
\N             1
dtype: int64

# on name basics

In [41]:
nbasics = pd.read_csv('imdb/name.basics.tsv', sep='\t', quoting=csv.QUOTE_NONE, na_values='\\N')

In [42]:
nbasics.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13986297 entries, 0 to 13986296
Data columns (total 6 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   nconst             13986297 non-null  object 
 1   primaryName        13986238 non-null  object 
 2   birthYear          632420 non-null    float64
 3   deathYear          237124 non-null    float64
 4   primaryProfession  11272084 non-null  object 
 5   knownForTitles     12404993 non-null  object 
dtypes: float64(2), object(4)
memory usage: 640.2+ MB


In [43]:
nbasics[nbasics.birthYear.isnull()].shape

(13353877, 6)

: 

# title akas

In [ ]:
# ['types'].unique() -> has '\x02' separators
# separator=bytearray.fromhex("02").decode('utf-8')
# null_corrected_df['correct_types'] = null_corrected_df['types'].str.replace(separator,";").str.split(";")

# separator=bytearray.fromhex("02").decode('utf-8')
# null_corrected_df['correct_attributes'] = null_corrected_df['attributes'].str.replace(separator,";").str.split(";")

In [5]:
!ls -la ..

total 1106884
drwxr-xr-x  4 lean lean      4096 Dec  1 04:22 .
drwxr-x--- 13 lean lean      4096 Nov 30 13:25 ..
drwxr-xr-x  4 lean lean      4096 Nov 30 11:36 MoviesRecommendedByLean
-rw-r--r--  1 lean lean 214322450 Sep 20  2019 genome_scores.csv
-rw-r--r--  1 lean lean     20363 Sep 20  2019 genome_tags.csv
drwxr-xr-x 12 lean lean      4096 Dec  1 04:22 imdb_dataset_article
-rw-r--r--  1 lean lean    539334 Sep 20  2019 link.csv
-rw-r--r--  1 lean lean   1493648 Sep 20  2019 movie.csv
-rw-r--r--  1 lean lean 204953792 Nov 29 20:46 movielens-20m-dataset.zip
-rw-r--r--  1 lean lean       291 Nov 29 20:45 movies.ipynb
-rw-r--r--  1 lean lean 690353377 Sep 20  2019 rating.csv
-rw-r--r--  1 lean lean  21725514 Sep 20  2019 tag.csv


# Trying DuckDB

In [2]:
import pandas as pd
import duckdb 
import os
import csv 
import gc
from tqdm.notebook import tqdm
from ydata_profiling import ProfileReport


db = duckdb.connect(database='imdb.duckdb') 

In [3]:
db.execute("SHOW TABLES").fetch_df() 

,name
0,name_basics_raw
1,title_akas_raw
2,title_basics_raw
3,title_crew_raw
4,title_episode_raw
5,title_principals_raw
6,title_ratings_raw


In [6]:
db.execute("select count(*) from name_basics_raw").fetch_df()

,count_star()
0,13986297


In [4]:
db.execute("SELECT * FROM title_basics_raw LIMIT 10;").fetch_df()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,NaN,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894.0,NaN,1.0,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894.0,NaN,1.0,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894.0,NaN,1.0,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894.0,NaN,45.0,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895.0,NaN,1.0,"Documentary,Short"


In [4]:
MAX_ROWS = 10_000_000 

In [ ]:
!mkdir -p profiling

In [ ]:
filenames = os.listdir('imdb')
tablenames = [f.replace('.','_') for f in filenames]
tablenames = [f.replace('_tsv','_raw') for f in tablenames]

for tablename in tablenames:
    size = db.execute(f"SELECT count(*) FROM {tablename}").fetch_df().values[0][0]
    if size > MAX_ROWS:
        sampledf = db.execute(f"""SELECT * FROM {tablename} USING 
                                  SAMPLE {MAX_ROWS} ROWS;""").fetch_df()
        profile = ProfileReport(sampledf, minimal=True, explorative=True)
    else:
        sampledf = db.execute(f"SELECT * FROM {tablename};").fetch_df()
        profile = ProfileReport(sampledf, explorative=True)
    
    profile.to_file(f"profiling/profile_report_{tablename}.html")

db.close()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## Gonna try profiling

In [15]:
!ls imdb

name.basics.tsv  title.basics.tsv  title.episode.tsv	 title.ratings.tsv
title.akas.tsv	 title.crew.tsv    title.principals.tsv


In [16]:
import os

# List all filenames in the imdb directory

print(filenames)

['title.ratings.tsv', 'title.principals.tsv', 'title.basics.tsv', 'title.akas.tsv', 'title.crew.tsv', 'title.episode.tsv', 'name.basics.tsv']


In [22]:
filenames

['title_ratings',
 'title_principals',
 'title_basics',
 'title_akas',
 'title_crew',
 'title_episode',
 'name_basics']

In [1]:
import duckdb

In [2]:
con = duckdb.connect('my_database.duckdb')
con.execute("SHOW TABLES").fetchdf()

,name
0,name_basics_raw
1,title_akas_raw
2,title_basics_raw
3,title_crew_raw
4,title_episode_raw
5,title_principals_raw
6,title_ratings_raw


In [3]:
con.close()

In [4]:
con.execute("select count(*) from title_akas_raw").fetchdf()

,count_star()
0,50662800


In [3]:
con.execute("DROP TABLE title_ratings_raw").fetchdf()

,Success


In [4]:
import duckdb
import pandas as pd
import os
import csv 
import gc
from tqdm.notebook import tqdm

# Connect to DuckDB (or create it if it doesn't exist)
con = duckdb.connect('my_database.duckdb')

In [7]:
# 0730

filenames = os.listdir('imdb')
#filenames = [f.replace('.','_') for f in filenames]
#filenames = [f.replace('_tsv','') for f in filenames]

# Load TSV files into DuckDB
for filename in tqdm(filenames):
    table_name = filename.replace('.','_').replace('_tsv','_raw')
    #df = pd.read_csv(f'imdb/{filename}', sep='\t', quoting=csv.QUOTE_NONE, na_values='\\N')
    #con.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")
    #del df 
    #gc.collect()

    #con.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM 'imdb/{filename}'")
    con.execute(f"""
         CREATE TABLE IF NOT EXISTS {table_name} AS
         SELECT *
            FROM read_csv('imdb/{filename}',
                delim = '\t',
                header = true,
                nullstr = '\\N',
                quote = '');
     """)
    # con.execute(f"""
    #     CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM 'imdb/{filename}'
    #     (DELIMITER '\t', NULL '\\N', QUOTE '', HEADER TRUE)
    #     """)

  0%|          | 0/7 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [1]:
1

1

In [1]:
import duckdb
import pandas as pd
import os
import csv 
import gc
from tqdm.notebook import tqdm
from tqdm import tqdm

In [2]:
filenames = os.listdir('imdb')
tablenames = [f.replace('.','_') for f in filenames]
tablenames = [f.replace('_tsv','_raw') for f in tablenames]

In [7]:
#import duckdb
import pandas as pd
import duckdb
import pandas as pd
import os
import csv 
import gc
from tqdm.notebook import tqdm
from ydata_profiling import ProfileReport

# Connect to DuckDB
#con = duckdb.connect('my_database.duckdb')

# Load data from DuckDB into a Pandas DataFrame
for filename in tqdm(filenames):
    #df = con.execute(f"SELECT * FROM {tablename}").fetchdf()
    df = pd.read_csv(f'imdb/{filename}', sep='\t', quoting=csv.QUOTE_NONE, na_values='\\N')

    # Generate the profile report
    profile = ProfileReport(df, title=f"YData Profiling Report for {filename}", explorative=True)
    profile.to_file(f"ydata_profiling_report_{filename}.html")
    del df 
    gc.collect()

  0%|          | 0/7 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:11<01:06, 11.09s/it]

: 

In [3]:
filenames

['title.ratings.tsv',
 'title.principals.tsv',
 'title.basics.tsv',
 'title.akas.tsv',
 'title.crew.tsv',
 'title.episode.tsv',
 'name.basics.tsv']

In [1]:
!ls -la imdb

total 8767756
drwxr-xr-x 2 lean lean       4096 Nov 30 11:39 .
drwxr-xr-x 4 lean lean       4096 Dec  1 07:31 ..
-rw-r--r-- 1 lean lean  860003015 Nov 30 11:39 name.basics.tsv
-rw-r--r-- 1 lean lean 2545837964 Nov 30 11:37 title.akas.tsv
-rw-r--r-- 1 lean lean  971215680 Nov 30 11:36 title.basics.tsv
-rw-r--r-- 1 lean lean  365321203 Nov 30 11:37 title.crew.tsv
-rw-r--r-- 1 lean lean  226685444 Nov 30 11:37 title.episode.tsv
-rw-r--r-- 1 lean lean 3982910975 Nov 30 11:38 title.principals.tsv
-rw-r--r-- 1 lean lean   26167479 Nov 30 11:38 title.ratings.tsv


In [2]:
!rm *.duckdb

In [ ]:
import duckdb
import pandas as pd
import os
import csv 
import gc
from tqdm.notebook import tqdm

# Connect to DuckDB
db = duckdb.connect('data.duckdb')

filenames = os.listdir('imdb')

# Configure chunk size (adjust based on available memory)
CHUNK_SIZE = 100000

for filename in filenames:
    table_name = filename.replace('.','_').replace('_tsv','_raw')
    
    # Get total lines for progress bar
    total_lines = sum(1 for _ in open(f'imdb/{filename}'))
    print(f"Total lines for {filename}: {total_lines}")
    total_chunks = total_lines // CHUNK_SIZE + 1

    # Create iterator for chunks
    chunks = pd.read_csv(f'imdb/{filename}', 
                        sep='\t',
                        quoting=csv.QUOTE_NONE, na_values='\\N',
                        chunksize=CHUNK_SIZE,
                        low_memory=False)

    # Create table on first chunk
    first_chunk = next(chunks)
    db.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} AS 
        SELECT * FROM first_chunk WHERE 1=0
    """)

    # Insert chunks with progress bar
    try:
        with tqdm(total=total_chunks) as pbar:
            # Insert first chunk
            db.execute(f"INSERT INTO {table_name} SELECT * FROM first_chunk")
            pbar.update(1)
            
            # Insert remaining chunks
            for chunk in chunks:
                db.execute(f"INSERT INTO {table_name} SELECT * FROM chunk")
                pbar.update(1)
    finally:
        chunks.close()
        del chunks
        gc.collect()            

db.close()

Total lines for title.ratings.tsv: 1505532


  0%|          | 0/16 [00:00<?, ?it/s]

Total lines for title.principals.tsv: 89457295


  0%|          | 0/895 [00:00<?, ?it/s]

Total lines for title.basics.tsv: 11272948


  0%|          | 0/113 [00:00<?, ?it/s]

Total lines for title.akas.tsv: 50662801


  0%|          | 0/507 [00:00<?, ?it/s]

In [1]:
1

1

In [ ]:
filename = "title.principals.tsv"

df = pd.read_csv(f'imdb/{filename}', sep='\t', quoting=csv.QUOTE_NONE, na_values='\\N')
print(df.shape)
# Generate the profile report
profile = ProfileReport(df, title=f"YData Profiling Report for {filename}", explorative=True)
profile.to_file(f"ydata_profiling_report_{filename}.html")
del df, profile
gc.collect()